In [1]:
import easyocr
import cv2
import os
import pandas as pd
from uuid import uuid4

# Rutas
input_image_path = "img/menu.png"  # tu imagen completa de menú
output_dir = "es_menu_train"       # carpeta destino

os.makedirs(output_dir, exist_ok=True)

reader = easyocr.Reader(['es'], gpu=False)
image = cv2.imread(input_image_path)
results = reader.readtext(image)

data = []
for idx, (bbox, text, _) in enumerate(results):
    pts = bbox
    x_coords = [int(pt[0]) for pt in pts]
    y_coords = [int(pt[1]) for pt in pts]
    x_min, x_max = max(min(x_coords), 0), min(max(x_coords), image.shape[1])
    y_min, y_max = max(min(y_coords), 0), min(max(y_coords), image.shape[0])

    cropped = image[y_min:y_max, x_min:x_max]
    file_id = f"line_{idx}_{uuid4().hex[:6]}.png"
    file_path = os.path.join(output_dir, file_id)
    cv2.imwrite(file_path, cropped)

    data.append({'filename': file_id, 'words': text})

df = pd.DataFrame(data)
df.to_csv(os.path.join(output_dir, "labels.csv"), index=False)


Using CPU. Note: This module is much faster with a GPU.
